In [0]:
# Step 1: Read CSV
oct_df = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
    header=True,
    inferSchema=True
)

# Step 2: Set Delta table path
delta_path = "/Volumes/workspace/ecommerce/delta/ecommerce_events"

# Step 3: Save as Delta table (overwrite if exists)
oct_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save(delta_path)

# Step 4: Import DeltaTable and Spark functions
from delta.tables import DeltaTable
from pyspark.sql.functions import col, row_number
from pyspark.sql.window import Window

# Step 5: Load Delta table
delta_table = DeltaTable.forPath(spark, delta_path)

# Step 6: Deduplicate source to avoid multiple source row errors
window_spec = Window.partitionBy(
    "user_session",
    "event_time",
    "product_id",
    "event_type"
).orderBy(col("event_time").desc())

updates_df = (
    oct_df
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)

# Step 7: Check count before merge
before_count = spark.read.format("delta").load(delta_path).count()
print("Before merge count of rows:", before_count)

# Step 8: Perform MERGE
delta_table.alias("t").merge(
    updates_df.alias("s"),
    """
    t.user_session = s.user_session AND
    t.event_time = s.event_time AND
    t.product_id = s.product_id AND
    t.event_type = s.event_type
    """
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

# Step 9: Check count after merge
after_count = spark.read.format("delta").load(delta_path).count()
print("After merge count of rows:", after_count)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8587327753182349>, line 15
      9 delta_path = "/Volumes/workspace/ecommerce/delta/ecommerce_events"
     11 # Step 3: Save as Delta table (overwrite if exists)
     12 oct_df.write \
     13     .format("delta") \
     14     .mode("overwrite") \
---> 15     .save(delta_path)
     17 # Step 4: Import DeltaTable and Spark functions
     18 from delta.tables import DeltaTable

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:703, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
    701     self.format(format)
    702 self._write.path = path
--> 703 _, _, ei = self._spark.client.execute_command(
    704     self._write.command(self._spark.client), self._write.observations
    705 )
    706 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/

In [0]:
from delta.tables import DeltaTable

# Load table
delta_table = DeltaTable.forPath(spark, delta_path)

# a) Show Delta history
spark.sql(f"DESCRIBE HISTORY delta.`{delta_path}`").show(truncate=False)

# b) Query an old version by version number
version_0_df = spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .load(delta_path)

print("Number of rows in version 0:", version_0_df.count())

# c) Query as of a timestamp
from datetime import datetime

# Example: all rows as of Jan 1, 2026
yesterday_df = spark.read.format("delta") \
    .option("timestampAsOf", "2026-01-12 15:20:23") \
    .load(delta_path)

print("Number of rows as of timestamp:", yesterday_df.count())


In [0]:
# Optimize Delta table with ZORDER by columns often queried
spark.sql(f"OPTIMIZE delta.`{delta_path}` ZORDER BY (event_type, user_id)")

In [0]:
spark.sql("""
VACUUM events_table RETAIN 168 HOURS
""")  # 7 days
